# E3. 使用 paddlenlp 和 fastNLP 实现中文文本情感分析

本篇教程属于 **fastNLP v1.0 tutorial 的 paddle examples 系列**。在本篇教程中，我们将为您展示如何使用 `paddlenlp` 自然语言处理库和 `fastNLP` 来完成比较简单的情感分析任务。

1. 基础介绍：飞桨自然语言处理库 ``paddlenlp`` 和语义理解框架 ``ERNIE``

2. 准备工作：使用 ``tokenizer`` 处理数据并构造 ``dataloader``

3. 模型训练：加载 ``ERNIE`` 预训练模型，使用 ``fastNLP`` 进行训练

### 1. 基础介绍：飞桨自然语言处理库 paddlenlp 和语义理解框架 ERNIE

#### 1.1 飞桨自然语言处理库 paddlenlp

``paddlenlp`` 是由百度以飞桨 ``PaddlePaddle`` 为核心开发的自然语言处理库，集成了多个数据集和 NLP 模型，包括百度自研的语义理解框架 ``ERNIE`` 。在本篇教程中，我们会以 ``paddlenlp`` 为基础，使用模型 ``ERNIE`` 完成中文情感分析任务。

In [8]:
import sys
sys.path.append("../")

import paddle
import paddlenlp
from paddlenlp.transformers import AutoTokenizer
from paddlenlp.transformers import AutoModelForSequenceClassification

print(paddlenlp.__version__)

2.3.3


#### 1.2 语义理解框架 ERNIE

``ERNIE（Enhanced Representation from kNowledge IntEgration）`` 是百度提出的基于知识增强的持续学习语义理解框架，至今已有 ``ERNIE 2.0``、``ERNIE 3.0``、``ERNIE-M``、``ERNIE-tiny`` 等多种预训练模型。``ERNIE 1.0`` 采用``Transformer Encoder`` 作为其语义表示的骨架，并改进了两种 ``mask`` 策略，分别为基于**短语**和**实体**（人名、组织等）的策略。在 ``ERNIE`` 中，由多个字组成的短语或者实体将作为一个统一单元，在训练的时候被统一地 ``mask`` 掉，这样可以潜在地学习到知识的依赖以及更长的语义依赖来让模型更具泛化性。

<img src="./figures/paddle-ernie-1.0-masking.png" align="center"></img>

<img src="./figures/paddle-ernie-1.0-masking-levels.png" align="center"></img>

``ERNIE 2.0`` 则提出了连续学习（``Continual Learning``）的概念，即首先用一个简单的任务来初始化模型，在更新时用前一个任务训练好的参数作为下一个任务模型初始化的参数。这样在训练新的任务时，模型便可以记住之前学习到的知识，使得模型在新任务上获得更好的表现。``ERNIE 2.0`` 分别构建了词法、语法、语义不同级别的预训练任务，并使用不同的 task id 来标示不同的任务，在共计16个中英文任务上都取得了SOTA效果。

<img src="./figures/paddle-ernie-2.0-continual-pretrain.png" align="center"></img>

``ERNIE 3.0`` 将自回归和自编码网络融合在一起进行预训练，其中自编码网络采用 ``ERNIE 2.0`` 的多任务学习增量式构建预训练任务，持续进行语义理解学习。其中自编码网络增加了知识增强的预训练任务。自回归网络则基于 ``Tranformer-XL`` 结构，支持长文本语言模型建模，并在多个自然语言处理任务中取得了SOTA的效果。

<img src="./figures/paddle-ernie-3.0-framework.png" align="center"></img>

接下来，我们将展示如何在 ``fastNLP`` 中使用基于 ``paddle`` 的 ``ERNIE 1.0`` 框架进行中文情感分析。

### 2. 使用 tokenizer 处理数据并构造 dataloader

#### 2.1 加载中文数据集 ChnSentiCorp

``ChnSentiCorp`` 数据集是由中国科学院发布的中文句子级情感分析数据集，包含了从网络上获取的酒店、电影、书籍等多个领域的评论，每条评论都被划分为两个标签：消极（``0``）和积极（``1``），可以用于二分类的中文情感分析任务。通过 ``paddlenlp.datasets.load_dataset`` 函数，我们可以加载并查看 ``ChnSentiCorp`` 数据集的内容。

In [9]:
from paddlenlp.datasets import load_dataset

train_dataset, val_dataset, test_dataset = load_dataset("chnsenticorp", splits=["train", "dev", "test"])
print("训练集大小：", len(train_dataset))
for i in range(3):
    print(train_dataset[i])

训练集大小： 9600
{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1, 'qid': ''}
{'text': '15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错', 'label': 1, 'qid': ''}
{'text': '房间太小。其他的都一般。。。。。。。。。', 'label': 0, 'qid': ''}


#### 2.2 处理数据

可以看到，原本的数据集仅包含中文的文本和标签，这样的数据是无法被模型识别的。同英文文本分类任务一样，我们需要使用 ``tokenizer`` 对文本进行分词并转换为数字形式的结果。我们可以加载已经预训练好的中文分词模型 ``ernie-1.0-base-zh``，将分词的过程写在函数 ``_process`` 中，然后调用数据集的 ``map`` 函数对每一条数据进行分词。其中：
- 参数 ``max_length`` 代表句子的最大长度；
- ``padding="max_length"`` 表示将长度不足的结果 padding 至和最大长度相同；
- ``truncation=True`` 表示将长度过长的句子进行截断。

至此，我们得到了每条数据长度均相同的数据集。

In [10]:
max_len = 128
model_checkpoint = "ernie-1.0-base-zh"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def _process(data):
    data.update(tokenizer(
        data["text"],
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
    ))
    return data

train_dataset.map(_process, num_workers=5)
val_dataset.map(_process, num_workers=5)
test_dataset.map(_process, num_workers=5)

print(train_dataset[0])

[2022-06-22 21:31:04,168] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'ernie-1.0-base-zh'.
[2022-06-22 21:31:04,171] [    INFO] - Already cached /remote-home/shxing/.paddlenlp/models/ernie-1.0-base-zh/vocab.txt


{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1, 'qid': '', 'input_ids': [1, 352, 790, 1252, 409, 283, 509, 5, 250, 196, 113, 10, 58, 518, 4, 9, 128, 70, 1495, 1855, 339, 293, 45, 302, 233, 554, 4, 544, 637, 1134, 774, 6, 494, 2068, 6, 278, 191, 6, 634, 99, 6, 2678, 144, 7, 149, 1573, 62, 12043, 661, 737, 371, 435, 7, 689, 4, 255, 201, 559, 407, 1308, 12043, 2275, 1110, 11, 19, 842, 5, 1207, 878, 4, 196, 198, 321, 96, 4, 16, 93, 291, 464, 1099, 10, 692, 811, 12043, 392, 5, 748, 1134, 10, 213, 220, 5, 4, 201, 559, 723, 595, 12043, 231, 112, 1114, 4, 7, 689, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

得到数据集之后，我们便可以将数据集包裹在 ``PaddleDataLoader`` 中，用于之后的训练。``fastNLP`` 提供的 ``PaddleDataLoader`` 拓展了 ``paddle.io.DataLoader`` 的功能，详情可以查看相关的文档。

In [11]:
from fastNLP.core import PaddleDataLoader
import paddle.nn as nn

train_dataloader = PaddleDataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = PaddleDataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = PaddleDataLoader(test_dataset, batch_size=1, shuffle=False)

### 3. 模型训练：加载 ERNIE 预训练模型，使用 fastNLP 进行训练

#### 3.1 使用 ERNIE 预训练模型

为了实现文本分类，我们首先需要定义文本分类的模型。``paddlenlp.transformers`` 提供了模型 ``AutoModelForSequenceClassification``，我们可以利用它来加载不同权重的文本分类模型。在 ``fastNLP`` 中，我们可以定义 ``train_step`` 和 ``evaluate_step`` 函数来实现训练和验证过程中的不同行为。

- ``train_step`` 函数在获得返回值 ``logits`` （大小为 ``(batch_size, num_labels)``）后计算交叉熵损失 ``CrossEntropyLoss``，然后将 ``loss`` 放在字典中返回。``fastNLP`` 也支持返回 ``dataclass`` 类型的训练结果，但二者都需要包含名为 **loss** 的键或成员。
- ``evaluate_step`` 函数在获得返回值 ``logits`` 后，将 ``logits`` 和标签 ``label`` 放在字典中返回。

这两个函数的参数均为数据集中字典**键**的子集，``fastNLP`` 会自动进行参数匹配然后输入到模型中。

In [12]:
import paddle.nn as nn

class SeqClsModel(nn.Layer):
    def __init__(self, model_checkpoint, num_labels):
        super(SeqClsModel, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_checkpoint,
            num_classes=num_labels,
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        logits = self.model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        return logits

    def train_step(self, input_ids, attention_mask, token_type_ids, label):
        logits = self(input_ids, attention_mask, token_type_ids)
        loss = nn.CrossEntropyLoss()(logits, label)
        return {"loss": loss}

    def evaluate_step(self, input_ids, attention_mask, token_type_ids, label):
        logits = self(input_ids, attention_mask, token_type_ids)
        return {'pred': logits, 'target': label}

model = SeqClsModel(model_checkpoint, num_labels=2)

[2022-06-22 21:31:15,577] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-1.0-base-zh'.
[2022-06-22 21:31:15,580] [    INFO] - Already cached /remote-home/shxing/.paddlenlp/models/ernie-1.0-base-zh/ernie_v1_chn_base.pdparams


#### 3.2 设置参数并使用 Trainer 开始训练

现在我们可以着手使用 ``fastNLP.Trainer`` 进行训练了。

首先，为了高效地训练 ``ERNIE`` 模型，我们最好为学习率指定一定的策略。``paddlenlp`` 提供的 ``LinearDecayWithWarmup`` 可以令学习率在一段时间内从 0 开始线性地增长（预热），然后再线性地衰减至 0 。在本篇教程中，我们将学习率设置为 ``5e-5``，预热时间为 ``0.1``，然后将得到的的 ``lr_scheduler`` 赋值给 ``AdamW`` 优化器。

其次，我们还可以为 ``Trainer`` 指定多个 ``Callback`` 来在基础的训练过程之外进行额外的定制操作。在本篇教程中，我们使用的 ``Callback`` 有以下三种：

- ``LRSchedCallback`` - 由于我们使用了 ``Scheduler``，因此需要将 ``lr_scheduler`` 传给该 ``Callback`` 以在训练中进行更新。
- ``LoadBestModelCallback`` - 该 ``Callback`` 会评估结果中的 ``'acc#accuracy'`` 值，保存训练中出现的正确率最高的模型，并在训练结束时加载到模型上，方便对模型进行测试和评估。

在 ``Trainer`` 中，我们还可以设置 ``metrics`` 来衡量模型的表现。``Accuracy`` 能够根据传入的预测值和真实值计算出模型预测的正确率。还记得模型中 ``evaluate_step`` 函数的返回值吗？键 ``pred`` 和 ``target`` 分别为 ``Accuracy.update`` 的参数名，在验证过程中 ``fastNLP`` 会自动将键和参数名匹配从而计算出正确率，这也是我们规定模型需要返回字典类型数据的原因。

``Accuracy`` 的返回值包含三个部分：``acc``、``total`` 和 ``correct``，分别代表 ``正确率``、 ``数据总数`` 和 ``预测正确的数目``，这让您能够直观地知晓训练中模型的变化，``LoadBestModelCallback`` 的参数 ``'acc#accuracy'`` 也正是代表了 ``accuracy`` 指标的 ``acc`` 结果。

在设定好参数之后，调用 ``run`` 函数便可以进行训练和验证了。

In [13]:
from fastNLP import LRSchedCallback, LoadBestModelCallback
from fastNLP import Trainer, Accuracy
from paddlenlp.transformers import LinearDecayWithWarmup

n_epochs = 2
num_training_steps = len(train_dataloader) * n_epochs
lr_scheduler = LinearDecayWithWarmup(5e-5, num_training_steps, 0.1)
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
)
callbacks = [
    LRSchedCallback(lr_scheduler, step_on="batch"),
    LoadBestModelCallback("acc#accuracy", larger_better=True, save_folder="fnlp-ernie"),
]
trainer = Trainer(
    model=model,
    driver="paddle",
    optimizers=optimizer,
    device=0,
    n_epochs=n_epochs,
    train_dataloader=train_dataloader,
    evaluate_dataloaders=val_dataloader,
    evaluate_every=60,
    metrics={"accuracy": Accuracy()},
    callbacks=callbacks,
)
trainer.run()

[21:31:16] INFO     Running evaluator sanity check for 2 batches.              ]8;id=4641;file://../fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=822054;file://../fastNLP/core/controllers/trainer.py#631\631]8;;\

---------------------------- Eval. results on Epoch:0, Batch:60 -----------------------------

{
  "acc#accuracy": 0.895833,
  "total#accuracy": 1200.0,
  "correct#accuracy": 1075.0
}

---------------------------- Eval. results on Epoch:0, Batch:120 ----------------------------

{
  "acc#accuracy": 0.8975,
  "total#accuracy": 1200.0,
  "correct#accuracy": 1077.0
}

...

{
  "acc#accuracy": 0.9375,
  "total#accuracy": 1200.0,
  "correct#accuracy": 1125.0
}

---------------------------- Eval. results on Epoch:1, Batch:300 ----------------------------

{
  "acc#accuracy": 0.941667,
  "total#accuracy": 1200.0,
  "correct#accuracy": 1130.0
}

[21:34:28] INFO     Loading best model from fnlp-ernie/2022-0 ]8;id=340364;file://../fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=763898;file://../fastNLP/core/callbacks/load_best_model_callback.py#111\111]8;;\
                    6-22-21_29_12_898095/best_so_far with                                    
                    acc#accuracy: 0.941667...                                                

[21:34:34] INFO     Deleting fnlp-ernie/2022-06-22-21_29_12_8 ]8;id=430330;file://../fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=508566;file://../fastNLP/core/callbacks/load_best_model_callback.py#131\131]8;;\
                    98095/best_so_far...                                                     

#### 3.3 测试和评估

现在我们已经得到了一个表现良好的 ``ERNIE`` 模型，接下来可以在测试集上测试模型的效果了。``fastNLP.Evaluator`` 提供了定制函数的功能。我们以 ``test_dataloader`` 初始化一个 ``Evaluator``，然后将写好的测试函数 ``test_batch_step_fn`` 传给参数 ``evaluate_batch_step_fn``，``Evaluate`` 在对每个 batch 进行评估时就会调用我们自定义的 ``test_batch_step_fn`` 函数而不是 ``evaluate_step`` 函数。在这里，我们仅测试 5 条数据并输出文本和对应的标签。

In [14]:
from fastNLP import Evaluator
def test_batch_step_fn(evaluator, batch):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    token_type_ids = batch["token_type_ids"]
    logits = model(input_ids, attention_mask, token_type_ids)
    predict = logits.argmax().item()
    print("text:", batch['text'])
    print("labels:", predict)

evaluator = Evaluator(
    model=model,
    dataloaders=test_dataloader,
    driver="paddle",
    device=0,
    evaluate_batch_step_fn=test_batch_step_fn,
)
evaluator.run(5)    

text: ['这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般']

labels: 0

text: ['怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片！开始
还怀疑是不是赠送的个别现象，可是后来发现每张DVD后面都有！真不知道生产商怎么想的，我想看的是猫
和老鼠，不是米老鼠！如果厂家是想赠送的话，那就全套米老鼠和唐老鸭都赠送，只在每张DVD后面添加一
集算什么？？简直是画蛇添足！！']

labels: 0

text: ['还稍微重了点，可能是硬盘大的原故，还要再轻半斤就好了。其他要进一步验证。贴的几种膜气
泡较多，用不了多久就要更换了，屏幕膜稍好点，但比没有要强多了。建议配赠几张膜让用用户自己贴。'
]

...

text: ['交通方便；环境很好；服务态度很好 房间较小']

labels: 1

text: ['不错，作者的观点很颠覆目前中国父母的教育方式，其实古人们对于教育已经有了很系统的体系
了，可是现在的父母以及祖父母们更多的娇惯纵容孩子，放眼看去自私的孩子是大多数，父母觉得自己的
孩子在外面只要不吃亏就是好事，完全把古人几千年总结的教育古训抛在的九霄云外。所以推荐准妈妈们
可以在等待宝宝降临的时候，好好学习一下，怎么把孩子教育成一个有爱心、有责任心、宽容、大度的人
。']

labels: 1

{}